In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/dataset.csv")
df.head()

,Filename,Text,Target
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,3
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",0
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,2
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,0
4,28353596_df.csv,A 47-year-old female patient presented progres...,2


In [4]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", device=0) #GPU

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [5]:
labels_dict = {'NEUTRAL':0,
                'LOW':1,
                'MODERATE':2,
                'HIGH':3}
def get_key(value,labels_dict):
  for key in labels_dict:
    if value == labels_dict[key]:
      return key
  return False


In [6]:
from tqdm import tqdm
final_list = []

candidate_labels = ["survivor", "death"]
for index in tqdm(range(len(df)),total=len(df)):
  text = df.iloc[index]["Text"]
  filename = df.iloc[index]["Filename"]
  results = classifier(text, candidate_labels)
  prediction = results['labels'][0]
  actual_label = get_key(df.iloc[index]["Target"],labels_dict)

  # print(filename,prediction,actual_label)
  final_list.append([filename,text,actual_label,prediction])


  5%|▌         | 10/200 [00:06<01:40,  1.88it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 200/200 [01:49<00:00,  1.82it/s]


In [7]:
final_df = pd.DataFrame(final_list,columns=['Filename','Text','API KEY Prediction','Predicted'])
final_df

,Filename,Text,API KEY Prediction,Predicted
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,HIGH,survivor
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",NEUTRAL,survivor
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,MODERATE,death
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,NEUTRAL,death
4,28353596_df.csv,A 47-year-old female patient presented progres...,MODERATE,survivor
...,...,...,...,...
195,21477357_df.csv,A 52-year-old man (body surface area: 1.3 m2) ...,HIGH,survivor
196,18258107_df.csv,"Here, we describe another case in a 60-year-ol...",LOW,survivor
197,19860007_df.csv,A 70-year-old man was referred to our hospital...,HIGH,survivor
198,18236639_df.csv,A 30-year-old female (65 kg) underwent rhinopl...,MODERATE,survivor


In [8]:
final_df.to_csv("/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/ZEROSHOT/pretrained_zeroshot-results.csv",index=False)

## Prediction of Death vs Survivour Dataset

In [10]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/MACROBATT/FINAL_SURVIVOR_DEATH/ZEROSHOT/pretrained_zero_shot_predicted_actual.csv")
df.head()

,Filename,Text,API KEY Prediction,Predicted,ACTUAL
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,HIGH,survivor,survivor
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",NEUTRAL,survivor,survivor
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,MODERATE,death,survivor
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,NEUTRAL,death,survivor
4,28353596_df.csv,A 47-year-old female patient presented progres...,MODERATE,survivor,survivor


In [11]:
from tqdm import tqdm
count_true = 0
true_positive = 0 # case predicted as survivour and in actual the case is survivour
false_negative = 0 # case predicted as survivour and in actual the case is dead
true_negative = 0 # case predicted as death and in actual the case is death
false_positive = 0 # case predicted as death and in actual the case is survivour

for index in tqdm(range(len(df[:200]))):
  if df.iloc[index].Predicted==df.iloc[index].ACTUAL:
    count_true +=1

  if df.iloc[index].Predicted=="survivor" and df.iloc[index].ACTUAL=="survivor":
    true_positive +=1
  elif df.iloc[index].Predicted=="survivor" and df.iloc[index].ACTUAL=="death":
    false_negative +=1
  elif df.iloc[index].Predicted=="death" and df.iloc[index].ACTUAL=="death":
    true_negative +=1
  elif df.iloc[index].Predicted=="death" and df.iloc[index].ACTUAL=="survivor":
    false_positive +=1







100%|██████████| 200/200 [00:00<00:00, 5379.45it/s]


In [12]:
# survivor
Precision = true_positive / (true_positive + false_positive)
Recall = true_positive / (true_positive + false_negative)
F1_Score = 2 * ((Precision*Recall)/(Precision+Recall))

In [13]:
print("Accuracy: ",count_true/len(df[:200]))
print("Precision: ",Precision)
print("Recall: ",Recall)
print("F1 Score: ",F1_Score)

Accuracy:  0.915
Precision:  0.9467455621301775
Recall:  0.9523809523809523
F1 Score:  0.9495548961424332


In [14]:
#death
Precision = true_negative/ (true_negative + false_negative)
Recall = true_negative / (true_negative + false_positive)
F1_Score = 2 * ((Precision*Recall)/(Precision+Recall))

In [15]:
print("Accuracy: ",count_true/len(df[:200]))
print("Precision: ",Precision)
print("Recall: ",Recall)
print("F1 Score: ",F1_Score)

Accuracy:  0.915
Precision:  0.7419354838709677
Recall:  0.71875
F1 Score:  0.7301587301587302
